In [1]:
import numpy as np
from interpol import Interpolation
from scipy.spatial import KDTree
import matplotlib.cm as cm
import meshplot as mp
import open3d as o3d
import networkx as nx
import trimesh as tm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh('../../data/humans/mesh/test_6890.off')

skeleton = []
with open('../../data/humans/centerline/test_52516.txt') as file:
    for line in file:
        contenidos = line.split(" ")
        vertices_en_rama = int(contenidos[0])
        vertices = np.reshape( np.array( contenidos[1:]).astype(np.float32), ((len(contenidos) - 1) // 3, 3) )
        skeleton.append( Interpolation(vertices) )
        

print(f"Hay {len(skeleton)} lineas")


Hay 21 lineas


In [11]:
centros = None
p = mp.plot( np.asarray(mesh.vertices), return_plot=True, shading={'point_size': 0.015, 'point_color':'#AA0000' })

cmap = cm.get_cmap('tab20')
for i, curve in enumerate(skeleton):
    c = '#'
    for x in (np.array(cmap( (i)/len(skeleton)) )[0:3] * 255).astype(np.uint32):
        h = hex(x)[2:]
        c += h if len(h) == 2 else '0' + h
    
    a = list(curve.evaluateFrom(curve.sampleByCurvature(pointsPerUnit=5)))
    
    centros = np.concatenate([ centros, np.asarray(a)]) if centros is not None else np.asarray(a)
    p.add_points( np.asarray(a), shading={'point_size': 0.1, 'point_color':c } )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0281797…

In [ ]:
len(centros)

## Matching

¿Qué pasa si hago el matching trivial? Es decir, para cada vértice me fijo que articulación está más cerca y me quedo con esa.

In [12]:
mesh = tm.load_mesh('../../data/humans/mesh/test_6890.off')
treeCenters = KDTree( centros )

a = treeCenters.query( np.asarray( mesh.triangles_center ), k=1 )

submeshes = { centro: [] for centro in range(len(centros))}

for idx, (point, nn) in enumerate( zip(np.asarray( mesh.triangles_center ), a[1] ) ):
    submeshes[nn].append( idx )

r = [len(i ) for i in submeshes.values()]
print( r, len(mesh.faces), sum(r))

[126, 78, 0, 0, 71, 93, 89, 126, 132, 76, 0, 150, 130, 0, 129, 84, 81, 123, 80, 86, 135, 142, 0, 262, 273, 162, 180, 199, 249, 0, 2269, 696, 241, 274, 43, 0, 0, 115, 116, 0, 268, 391, 882, 0, 130, 73, 0, 132, 115, 233, 0, 0, 0, 451, 429, 228, 589, 247, 200, 141, 147, 298, 251, 0, 0, 0, 257, 378, 888, 0, 38, 0] 13776 13776


In [13]:
recs = []
for m in submeshes.values():
    t = mesh.submesh( [m], append=True ) 
    if isinstance(t, tm.Trimesh):
        recs.append( t )
    else:
        recs.append( tm.Trimesh() )
    #newMesh = o3d.geometry.TriangleMesh( mesh  )
    #newMesh.remove_vertices_by_mask( [ i not in m for i in range(len(mesh.vertices)) ])
    

In [14]:
p3 = mp.plot( np.zeros((1,3)) , return_plot=True)
cmap = cm.get_cmap('flag')
for i, m in enumerate(recs):
    
    if len(np.asarray(m.vertices)) != 0:
        p3.add_mesh( np.asarray(m.vertices), np.asarray(m.faces), c=np.array(cmap((i)/len(recs)))[0:3] )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Usando optimización

Introduzco una función de pérdida L.

Quiero que L sea más grande si:
- Un matcheo tiene más de una componente conexa
- La distancia entre el centro de masa y la articulación es grande
- ...

In [ ]:
def L( submeshes, articulaciones, w ):

    Lcc = lambda s: sum( [ np.exp( (nx.number_connected_components( tm.graph.vertex_adjacency_graph(s_i) ) - 1) * 1 ) - 1 for s_i in s if len(s_i.vertices) != 0] )

    Lcm = lambda s, a: sum( [ np.linalg.norm( np.average(s_i.triangles_center, axis=0) - a_i ) for s_i, a_i in zip(s, a) if len(s_i.vertices) != 0 ])

    return w[0] * Lcc(submeshes) + w[1] * Lcm(submeshes, articulaciones)

In [ ]:
L( recs, centros, (1,1) )

# Problema

Quiero asignarle triangulos a cada articulación de tal forma que:
- La asignación sea una submalla que no tenga huecos ni multiples componentes
- Que el centro de masa de esta submalla sea lo más cercano a la articulación posible
- 

In [9]:
beg, end = [], []
## EL PRIMER VERTICE QUE APARECE ES DEL ESQUELETO
with open('../../data/humans/centerline/faust_15070.polylines.txt') as file:
    for line in file:
        s = line.split(' ')
        beg.append( np.array(s[1:4]).astype(np.float32) )
        end.append( np.array(s[4:]).astype(np.float32) ) 

mesh = o3d.io.read_triangle_mesh('../../data/humans/mesh/test_6890.off')
p = mp.plot( np.asarray(mesh.vertices), return_plot=True, shading={'point_size': 0.01, 'point_color':'#AA0000' })
p.add_lines( np.array(beg), np.array(end))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0281797…

1

In [15]:
len(np.unique(beg, axis=0))

561